In [1]:
import sys, os, string
import numpy as np
from datetime import datetime

In [2]:
# Perfect phylogeny

def perfectPhylogeny(m):
    m_prime = get_m_prime(m)
    (k, features) = get_k_matrix_features(m_prime)
    locations = []
    for feature in features:
        present_at = set([])
        for k_i in k:
            [ present_at.add(loc_list) for loc_list in list(np.where(k_i==feature)[0]) ]
        locations.append(list(present_at))

    loc_test = np.array([len(loc_list)>1 for loc_list in locations])
    if np.any(loc_test):
#         print('No phylogeny found!')
        return False
    else:    
#         print('Success! Found phylogeny!')
        return True
    
def get_m_prime(m):
    # rotate M for convenient iteration
    m_prime = np.rot90(m)

    # keep only unique combinations
    m_prime = np.unique(list(map(lambda x: '.'.join(map(str, x)), m_prime)))
    m_prime = np.array(list(map(lambda x: list(map(int, x.split('.'))), m_prime)))

    # count binary score of columns
    binary_strings = []
    for col in m_prime:
        col_string = '0b'+''.join(map(str, col))
        binary_strings.append(int(col_string, 2))

    # sort by binary score
    order = np.argsort(binary_strings)[::-1]
    m_prime = m_prime[order] 

    m_prime = np.rot90(m_prime[:,:])[::-1] #rotate again
    return m_prime

def get_k_matrix_features(m_prime):
    ncol = len(m_prime[0])
    k = np.empty([0, ncol], dtype='|S15')
    features = np.array(list(string.ascii_lowercase[:ncol]), dtype='|S15')

    for m in m_prime:
        row_feats = features[m!=0] #features in the row
        mrow = np.zeros(ncol, dtype='|S15')
        mrow.fill('0')

        for idx,feature in enumerate(row_feats):
            mrow[idx] = feature

        n_feat = len(row_feats)    
        if n_feat < ncol: 
            mrow[n_feat]='#'

        k = np.append(k,[mrow],axis=0)

    return (k, features)

In [3]:
m = np.array([[1, 1, 0, 1, 0, 1, 1, 1, 1, 1],
              [0, 1, 0, 1, 0, 0, 0, 1, 0, 1],
              [0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

perfectPhylogeny(m)

True

In [4]:
import random

from deap import base
from deap import creator
from deap import tools

In [5]:
# I - input matrix
# E - expected matrix
# i, e - elements from corresponding matrix
# alpha - false negative
# beta - false positive

def P(i, e, alpha=0, beta=0):
    if i == e == 0: return 1 - beta
    if i == 1 and e == 0: return beta
    if i == e == 1: return 1 - alpha
    if i == 0 and e == 1: return alpha
    
vp = np.vectorize(P)

def likelihood(I, E):
    return np.sum(vp(I, E))

In [12]:
def model(n, m, INPUT_MATRIX):
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)
    
    toolbox = base.Toolbox()

    # Attribute generator 
    #                      define 'attr_bool' to be an attribute ('gene')
    #                      which corresponds to integers sampled uniformly
    #                      from the range [0,1] (i.e. 0 or 1 with equal
    #                      probability)
    toolbox.register("attr_bool", random.randint, 0, 1)

    # Structure initializers
    #                         define 'individual' to be an individual
    #                         consisting of 100 'attr_bool' elements ('genes')
    toolbox.register("individual", tools.initRepeat, creator.Individual, 
        toolbox.attr_bool, n*m)

    # define the population to be a list of individuals
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    def fitness(individual):
        return (likelihood(individual, INPUT_MATRIX),)

    def feasible(individual):
        """Feasability function for the individual. Returns True if feasible False
        otherwise."""
        i_matrix = np.array(individual).reshape(n, m)
        return perfectPhylogeny(i_matrix)

    #----------
    # Operator registration
    #----------
    # register the goal / fitness function
    toolbox.register("evaluate", fitness)
    toolbox.decorate("evaluate", tools.DeltaPenality(feasible, -n*m/2))

    # register the crossover operator
    toolbox.register("mate", tools.cxTwoPoint)

    # register a mutation operator with a probability to
    # flip each attribute/gene of 0.05
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)

    # operator for selecting individuals for breeding the next
    # generation: each individual of the current generation
    # is replaced by the 'fittest' (best) of three individuals
    # drawn randomly from the current generation.
    toolbox.register("select", tools.selTournament, tournsize=3)
    
    return toolbox

In [13]:
def main(INPUT_MATRIX):
    random.seed(64)
    
    n, m = INPUT_MATRIX.shape
    toolbox = model(n, m, INPUT_MATRIX.reshape(n*m))

    # create an initial population of 300 individuals (where
    # each individual is a list of integers)
    pop = toolbox.population(n=300)

    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    CXPB, MUTPB = 0.5, 0.2
    
    print("Start of evolution")
    
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    
    print("  Evaluated %i individuals" % len(pop))

    # Extracting all the fitnesses of 
    fits = [ind.fitness.values[0] for ind in pop]

    # Variable keeping track of the number of generations
    g = 0
    
    # Begin the evolution
    while max(fits) < n*m*0.8 and g < 100:
        # A new generation
        g = g + 1
        print("-- Generation %i --" % g)
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
    
        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # cross two individuals with probability CXPB
            if random.random() < CXPB:
                toolbox.mate(child1, child2)

                # fitness values of the children
                # must be recalculated later
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            # mutate an individual with probability MUTPB
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values
    
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        print("  Evaluated %i individuals" % len(invalid_ind))
        
        # The population is entirely replaced by the offspring
        pop[:] = offspring
        
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
    
    print("-- End of (successful) evolution --")
    
    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))

In [14]:
INPUT_MATRIX = np.array([[1, 0, 0],
[1, 0, 1],
[0, 1, 1],
                        [0, 1, 1],
                        [1, 0, 1],
                        [0, 0, 1],
                        [0, 1, 0]])

main(INPUT_MATRIX)

Start of evolution
  Evaluated 300 individuals
-- Generation 1 --
  Evaluated 177 individuals
  Min -10.5
  Max 16.0
  Avg -5.971666666666667
  Std 8.838176502474303
-- Generation 2 --
  Evaluated 167 individuals
  Min -10.5
  Max 16.0
  Avg -1.4216666666666666
  Std 10.835806256214727
-- Generation 3 --
  Evaluated 182 individuals
  Min -10.5
  Max 17.0
  Avg 3.3516666666666666
  Std 11.069424731614959
-- End of (successful) evolution --
Best individual is [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1], (17.0,)
